In [64]:
SEARCH_URL = 'https://stardewvalleywiki.com/mediawiki/index.php?action=opensearch&search='

In [60]:
def was_result_found(soup):
    return soup.find(id="firstHeading").text != "Search results"

In [71]:
def find_infobox_section(soup, name):

    infobox = soup.find(id="infoboxtable")

    if not infobox:
        return f'No infobox found for {name}'

    table_rows = infobox.findAll('tr')

    for row in table_rows[3:]:
        
        section = row.find(id='infoboxsection')

        if section and section.text.strip() == f'{name}:':
            return row.find(id='infoboxdetail')

    return None


In [100]:
def section_to_readable(info):
    return ', '.join(map(lambda item: item.text, info.findAll('a')))


# Gifting routes

## Birthday

In [76]:
from bs4 import BeautifulSoup
import requests


def find_birthday(name):
    url = f'{SEARCH_URL}{name}'

    page = requests.get(url).text
    soup = BeautifulSoup(page)

    # was the search succesful?
    if not was_result_found(soup):
        return f'{name} was not found on the wiki'

    # try and find birthday info in info box
    birthday_info = find_infobox_section(soup, 'Birthday')

    # if we couldn't find birthday info in the infobox, return error message
    if not birthday_info:
        return f'Could not find info about the birthday of {name}'

    # get info about birthday
    return birthday_info.text.strip()


find_birthday('George')


'Fall 24'

## Love Gifts

In [101]:
def find_best_gifts(name):
    url = f'{SEARCH_URL}{name}'

    page = requests.get(url).text
    soup = BeautifulSoup(page)

    # was the search succesful?
    if not was_result_found(soup):
        return f'{name} was not found on the wiki'

    best_gifts_info = find_infobox_section(soup, 'Best Gifts')

    if not best_gifts_info:
        return f'Could not find info about the best gifts for {name}'

    # return human readable form of gift info
    return section_to_readable(best_gifts_info)

find_best_gifts('Evelyn')


'Beet, Chocolate Cake, Diamond, Fairy Rose, Stuffing, Tulip'

## Universal Loves

In [102]:
def find_universal_loves():
    url = f'{SEARCH_URL}List of All Gifts'

    page = requests.get(url).text
    soup = BeautifulSoup(page)

    # find the universal loves section on the page
    universal_loves_info = soup.findAll('tbody')[1].findAll('tr')[1].findAll('td')[2]

    # return human readable form of universal loves info
    return section_to_readable(universal_loves_info)

find_universal_loves()

"Golden Pumpkin, Magic Rock Candy, Pearl, Prismatic Shard, Rabbit's Foot"

# Item route

In [138]:
def replace_dots_with_commas(string):
    words = string.split('•')
    return ', '.join(map(lambda word: word.strip(), words))
    

## Info about fish: season/time/location

In [140]:
def find_fish_info(fish):
    url = f'{SEARCH_URL}{fish}'

    page = requests.get(url).text
    soup = BeautifulSoup(page)

    # was the search succesful?
    if not was_result_found(soup):
        return f'{fish} was not found on the wiki'

    season_info = find_infobox_section(soup, 'Season')
    time_info = find_infobox_section(soup, 'Time of Day')
    location_info = find_infobox_section(soup, 'Found in')

    if not (season_info and time_info and location_info):
        return f'Could not find all fishing information about {fish}'

    season_info = replace_dots_with_commas(season_info.text)
    time_info = time_info.text.strip()
    location_info = replace_dots_with_commas(location_info.text)

    return {'seasons': season_info, 'time': time_info, 'locations': location_info}

find_fish_info('Sardine')


{'seasons': 'Spring, Fall, Winter', 'time': '6am – 7pm', 'locations': 'Ocean'}

## Info about crops: season/growth timeline

In [144]:
def find_crop_info(crop):
    url = f'{SEARCH_URL}{crop}'

    page = requests.get(url).text
    soup = BeautifulSoup(page)

    # was the search succesful?
    if not was_result_found(soup):
        return f'{crop} was not found on the wiki'

    growth_time_info = find_infobox_section(soup, 'Growth Time')
    seasons_info = find_infobox_section(soup, 'Season') or find_infobox_section(soup, 'Harvest Season')

    if not (growth_time_info and seasons_info):
        return f'Could not find all agriculture information about {crop}'

    growth_time_info = growth_time_info.text.strip()
    seasons_info = replace_dots_with_commas(seasons_info.text)

    return {'growth_time': growth_time_info, 'seasons': seasons_info}

find_crop_info('Alex')


'Could not find all agriculture information about Alex'